In [1]:
# imports
import os
import pandas as pd
import json
import requests
import copy
import re
import numpy as np

In [2]:
#Loading in the bike station data, parsed in the previous step
bike_station_info = pd.read_csv('all_bike_stations_toronto.csv')

In [3]:
four_square_key_new = os.environ['FOUR_SQUARE_API_KEY']

# Foursquare

In [4]:
#The bike staions dataframe
bike_station_info

,Unnamed: 0,bikes_no,longitude,latitude
0,0,18,-79.319796,43.665269
1,1,3,-79.445947,43.671420
2,2,20,-79.317693,43.666224
3,3,10,-79.376716,43.653236
4,4,10,-79.380288,43.663722
...,...,...,...,...
669,669,7,-79.219952,43.735967
670,670,0,-79.287660,43.715466
671,671,4,-79.437461,43.642904
672,672,1,-79.268510,43.706880


Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

### Finding the bars withing a 1000M radius of a single latitude, later we will do this for every bikestop

In [5]:
url = "https://api.foursquare.com/v3/places/search"

params={
    "query":'bar',
    "fsq_category_ids":[13006],
    "ll":'43.665269,-79.319796',
    'radius':1000,
    "sort":"DISTANCE",
    "limit":'50'
}

headers={
    "Accept":"application/json",
    "Authorization":four_square_key_new
}

response = requests.request("GET", url, params=params, headers=headers)

In [6]:
data = response.json()

In [7]:
data 

{'results': [{'fsq_id': 'bf8257b1d8d943ca4e046166',
   'categories': [{'id': 13322,
     'name': 'Peruvian Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/peruvian_',
      'suffix': '.png'}},
    {'id': 13338,
     'name': 'Seafood Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 108,
   'geocodes': {'drop_off': {'latitude': 43.665831, 'longitude': -79.318831},
    'main': {'latitude': 43.665973, 'longitude': -79.318863},
    'roof': {'latitude': 43.665973, 'longitude': -79.318863}},
   'link': '/v3/places/bf8257b1d8d943ca4e046166',
   'location': {'address': '1560 Queen St E',
    'country': 'CA',
    'formatted_address': '1560 Queen St E, Toronto ON M4L 1E9',
    'locality': 'Toronto',
    'postcode': 'M4L 1E9',
    'region': 'ON'},
   'name': 'T.O. Ceviche Bar',
   'related_places': {}},
  {'fsq_id': 'e03675b4909543f50a0945d9',
   'categories': [{

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
info_parsed = {'name':[],'address':[],'distance':[]}
for x in data['results']:
    if re.search(r'Barber', x['name']) is None:
        print('correct')
        print('Name: ' + x['name'] + ' || Address: ' + x['location']['address'] + ' || Distance from bike stop: ' + str(x['distance']))
        info_parsed['name'].append(x['name'])
        info_parsed['address'].append(x['location']['address'])
        info_parsed['distance'].append(x['distance'])

correct
Name: T.O. Ceviche Bar || Address: 1560 Queen St E || Distance from bike stop: 108
correct
Name: Chalet Beauty Bar || Address: 1562 Queen St E || Distance from bike stop: 108
correct
Name: Q Club Restaurant Bar || Address: 1574 Queen St E || Distance from bike stop: 145
correct
Name: Rorschach Brewing || Address: 1001 Eastern Ave || Distance from bike stop: 194
correct
Name: Murphy's Law Irish Pub Inc || Address: 1702 Queen St E || Distance from bike stop: 617
correct
Name: The Burren Pub || Address: 1301 Queen St E || Distance from bike stop: 688
correct
Name: Betty's East || Address: 1301 Queen St E || Distance from bike stop: 697
correct
Name: Lloyd's on Queen || Address: 1298 Queen St E || Distance from bike stop: 702
correct
Name: Hasting Snack Bar || Address: 5 Hastings Ave || Distance from bike stop: 759
correct
Name: Triple a Bar || Address: 1276 Queen St E || Distance from bike stop: 770
correct
Name: Black Ponny Cafe & Bar || Address: 1481 Gerrard St E || Distance fro

Put your parsed results into a DataFrame

In [9]:
four_square_parsed_df = pd.DataFrame(info_parsed)

In [10]:
four_square_parsed_df

,name,address,distance
0,T.O. Ceviche Bar,1560 Queen St E,108
1,Chalet Beauty Bar,1562 Queen St E,108
2,Q Club Restaurant Bar,1574 Queen St E,145
3,Rorschach Brewing,1001 Eastern Ave,194
4,Murphy's Law Irish Pub Inc,1702 Queen St E,617
5,The Burren Pub,1301 Queen St E,688
6,Betty's East,1301 Queen St E,697
7,Lloyd's on Queen,1298 Queen St E,702
8,Hasting Snack Bar,5 Hastings Ave,759
9,Triple a Bar,1276 Queen St E,770


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
#os.environ['YELP_API_KEY'] = "GATTyxS-vK1yP_oEBm8_o0rUWiC2U1VJV83enLPf7FfIZU-VgcjOGGVNqa8eh-4c0cDOgQztFH_BE4yKDMxcxJE0IdHj6VDPa3w8otSSVn3LzMEH-4jGKjVe_CrGZHYx"
YELP_KEY = os.environ['YELP_API_KEY']

In [12]:
#Yelp Client ID: gBKxfJ7XANubNFtW-fwgBg
url = "https://api.yelp.com/v3/businesses/search"

headers = {
    "Authorization": "Bearer " + YELP_KEY,
    "Accept":"application/json"
}

params = {
    "latitude": 43.665269,  # Replace with your actual latitude
    "longitude": -79.319796,  # Replace with your actual longitude
    "categories":['bars'],
    "radius":1000,
    "sort_by":"distance"
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

# Now, `data` contains the businesses around your specified coordinates!

In [16]:
data

{'businesses': [{'id': 'ILSWNor1STnYyDLsraEY2w',
   'alias': 'betty-s-east-toronto',
   'name': 'Betty’s East',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ev6eZscg2qOKFzXt-FFP3w/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/betty-s-east-toronto?adjust_creative=gBKxfJ7XANubNFtW-fwgBg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=gBKxfJ7XANubNFtW-fwgBg',
   'review_count': 3,
   'categories': [{'alias': 'salad', 'title': 'Salad'},
    {'alias': 'chicken_wings', 'title': 'Chicken Wings'},
    {'alias': 'beerbar', 'title': 'Beer Bar'}],
   'rating': 4.5,
   'coordinates': {'latitude': 43.663455, 'longitude': -79.327992},
   'transactions': [],
   'location': {'address1': '1301 Queen Street E',
    'address2': '',
    'address3': None,
    'city': 'Toronto',
    'zip_code': 'M4L 1C2',
    'country': 'CA',
    'state': 'ON',
    'display_address': ['1301 Queen Street E',
     'Toronto, ON M4L 1C2',
     'Canada']},
   'phone': '+1416

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [17]:
yelp_parsed_info = {'name':[],'rating':[],'address':[],'latitude':[],'longitude':[],'distance':[]}
for x in data['businesses']:
    name_to_add = x['name']
    yelp_parsed_info['name'].append(name_to_add)
    
    rating_to_add = x['rating']
    yelp_parsed_info['rating'].append(rating_to_add)
    
    address_to_add = x['location']['address1']
    yelp_parsed_info['address'].append(address_to_add)
    
    latitude_to_add = x['coordinates']['latitude']
    yelp_parsed_info['latitude'].append(latitude_to_add)
    
    longitude_to_add = x['coordinates']['longitude']
    yelp_parsed_info['longitude'].append(longitude_to_add)
    
    distance_to_add = x['distance']
    yelp_parsed_info['distance'].append(distance_to_add)

Put your parsed results into a DataFrame

In [19]:
yelp_df = pd.DataFrame(yelp_parsed_info)
#Dropping row because it was a cornerstore, not what we were looking for
yelp_df = yelp_df.drop([8]).reset_index(drop=True)
yelp_df = yelp_df.sort_values(by=['distance'])
yelp_df

,name,rating,address,latitude,longitude,distance
0,Betty’s East,4.5,1301 Queen Street E,43.663455,-79.327992,689.778721
1,Lloyd's On Queen,5.0,1298 Queen St E,43.663801,-79.328356,707.598820
2,Boardwalk Place,2.5,1675 Lake Shore Boulevard E,43.663010,-79.310898,758.567399
3,Baldev Paan Cold Drink House,3.0,1399 Gerrard Street E,43.671710,-79.323520,772.512252
4,Two Headed Dog,2.5,1469 Gerrard Street E,43.672358,-79.320782,792.227554
5,The Thirsty Loon,4.0,1489 Gerrard Street E,43.672394,-79.319906,792.332218
6,Lake Inez,4.5,1471 Gerrard Street E,43.672340,-79.320640,794.661955
7,Black Pony,4.5,1481 Gerrard Street E,43.672410,-79.320400,805.518073
8,Jenni Place,3.0,1338 Gerrard Street E,43.671800,-79.324810,829.759140
9,Beach Club,3.0,1681 Lake Shore Boulevard E,43.663207,-79.310013,839.312073


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The Yelp Fusion API provided me with much more data. I was able to collect information about the the ratings at each restaurant. The foursquare api only provided me with the location.

Get the top 10 restaurants according to their rating

In [21]:
#Print the top 10 restaurants according to their rating
top_rated_df = yelp_df.sort_values(by=['rating'],ascending=False).reset_index(drop=True)
top_rated_df.head(10)

,name,rating,address,latitude,longitude,distance
0,Lloyd's On Queen,5.0,1298 Queen St E,43.663801,-79.328356,707.598820
1,Betty’s East,4.5,1301 Queen Street E,43.663455,-79.327992,689.778721
2,Lake Inez,4.5,1471 Gerrard Street E,43.672340,-79.320640,794.661955
3,Black Pony,4.5,1481 Gerrard Street E,43.672410,-79.320400,805.518073
4,Le Conciliabule,4.5,1300 Gerrard Street E,43.671365,-79.327225,903.609078
5,The Thirsty Loon,4.0,1489 Gerrard Street E,43.672394,-79.319906,792.332218
6,Shamrock Bowl and Restobar,4.0,280 Coxwell Avenue,43.673290,-79.319790,896.914859
7,JP Restaurant,3.5,270 Coxwell Avenue,43.673030,-79.319550,868.716398
8,The Duke Live,3.5,1225 Queen Street E,43.663149,-79.330307,877.705944
9,Baldev Paan Cold Drink House,3.0,1399 Gerrard Street E,43.671710,-79.323520,772.512252


In [22]:
yelp_df.to_csv('yelp_poi_info.csv')